In [89]:
import torch
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [90]:
import pandas as pd
import numpy as np
import os
import torch
from torch import nn
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

output_nums=4


# 修改后的createXY函数
def createXY(dataset: pd.DataFrame, n_past: int, n_future: int, column_target: str):
    dataX, dataY = [], []
    for i in range(n_past, len(dataset) - n_future + 1):
        dataX.append(dataset.iloc[i - n_past:i].values)
        dataY.append(dataset.iloc[i:i + n_future][column_target].values)
    return np.array(dataX), np.array(dataY)

# 修改后的process_files函数
def process_files(columns_all, column_target, folder_path, n_past=1, n_future=1):
    all_dataX, all_dataY = np.array([]), np.array([])
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            series = pd.read_csv(file_path)
            single_dataset = series[columns_all]
            dataX, dataY = createXY(single_dataset, n_past, n_future, column_target)
            all_dataX = np.vstack([all_dataX, dataX]) if all_dataX.size else dataX
            all_dataY = np.vstack([all_dataY, dataY]) if all_dataY.size else dataY
    return all_dataX, all_dataY


columns_all=['CGM (mg / dl)','insulin','carbohydrate','protein','fat','cellulose','hour_sin','hour_cos','CGM_diff']
# columns_all = ['CGM (mg / dl)','CSII - basal insulin (Novolin R, IU / H)']
column_target = ['CGM (mg / dl)']
folder_path = '../diabetes_datasets/T2'

# 使用n_past=8, n_future=4调用process_files
n_past=8
dataX, dataY = process_files(columns_all, column_target, folder_path, n_past=n_past, n_future=output_nums)



In [92]:
dataY_for_train=dataY[:,0,:].reshape(-1,1,1)

In [93]:
dataX.shape,dataY.shape,dataY_for_train.shape

((111142, 8, 9), (111142, 4, 1), (111142, 1, 1))

In [94]:
dataY[0],dataY_for_train[0]

(array([[68.4],
        [75.6],
        [61.2],
        [48.6]]),
 array([[68.4]]))

In [99]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch import nn, optim
from torch.utils.data import Subset

# 假设dataX和dataY是你的数据
# 将它们转换为PyTorch张量，这里假设它们已经是Tensor或者从Numpy转换过来的
dataX_tensor = torch.tensor(dataX, dtype=torch.float32)
dataY_tensor = torch.tensor(dataY_for_train, dtype=torch.float32)

# 创建TensorDataset对象
dataset = TensorDataset(dataX_tensor, dataY_tensor)

# 其余拆分数据集、创建DataLoader对象的代码与之前相同
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - (train_size + val_size)


# # 随机拆分数据集
# train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

assert train_size + val_size + test_size == len(dataset)
# 按照顺序划分数据集
train_dataset = Subset(dataset, range(0, train_size))
val_dataset = Subset(dataset, range(train_size, train_size + val_size))
test_dataset = Subset(dataset, range(train_size + val_size, len(dataset)))


batch_size = 16
# 创建DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
feature_nums = dataX.shape[2]


In [102]:
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_dim=feature_nums, output_dim=1, d_model=4, nhead=2, num_encoder_layers=1, dropout=0.2):
        super(TransformerEncoderModel, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.d_model = d_model
        self.nhead = nhead
        self.num_encoder_layers = num_encoder_layers
        
        # 使用线性层代替嵌入层
        self.linear_input = nn.Linear(self.input_dim, self.d_model)

        # 多头自注意力层
        encoder_layers = nn.TransformerEncoderLayer(d_model=self.d_model, nhead=self.nhead, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=self.num_encoder_layers)

        # 输出层
        self.linear_out = nn.Linear(self.d_model, self.output_dim)

    def forward(self, x):
        # 通过线性层
        x = self.linear_input(x)

        # 通过Transformer编码器
        x = self.transformer_encoder(x)

        # 取最后一个时间步的输出
        x = x[:, -1, :]

        # 通过输出层
        x = self.linear_out(x)

        return x

### 从零训练和测试

In [97]:
# model = TransformerEncoderModel().to(device)
# criterion = nn.L1Loss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # 早停法参数
# best_val_loss = float('inf')
# patience = 10
# patience_counter = 0
# max_epochs = 5000

# for epoch in range(max_epochs):
#     train_loss = 0
#     model.train()
    
#     # Training loop
#     for data, target in train_loader:
#         data, target = data.to(device), target.to(device)
#         target = target.squeeze(-1)  # 调整目标尺寸
#         optimizer.zero_grad()
#         output = model(data)
#         loss = criterion(output, target)
#         train_loss += loss.item()
#         loss.backward()
#         optimizer.step()
    
#     # 每个epoch后，在验证集上评估模型
#     model.eval()
#     with torch.no_grad():
#         val_loss = 0
#         for data, target in val_loader:
#             data, target = data.to(device), target.to(device)
#             target = target.squeeze(-1)  # 调整目标尺寸
#             output = model(data)
#             val_loss += criterion(output, target).item()
    
#     train_loss /= len(train_loader)
#     val_loss /= len(val_loader)

#     print(f'Epoch {epoch}: Training Loss: {train_loss}, Validation Loss: {val_loss}')

#     # 早停法逻辑
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         patience_counter = 0
#         # 保存最好的模型状态
#         best_model_state = model.state_dict()
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print(f'Early stopping triggered after {epoch} epochs.')
#         model.load_state_dict(best_model_state)
#         break



In [98]:
# import torch

# # 保存模型的状态字典到文件
# torch.save(model.state_dict(), '../model/T2/transformerEncoderMultiStep.pth')


In [103]:
dataY_tensor = torch.tensor(dataY, dtype=torch.float32)
# 创建TensorDataset对象
dataset = TensorDataset(dataX_tensor, dataY_tensor)
# 其余拆分数据集、创建DataLoader对象的代码与之前相同
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
assert train_size + val_size + test_size == len(dataset)
test_dataset = Subset(dataset, range(train_size + val_size, len(dataset)))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# # 加载最佳模型状态
# model.load_state_dict(best_model_state)
# criterion_mae=nn.L1Loss()
# # 在测试集上验证模型并输出预测标签
# model.eval()
# test_loss = 0
# test_loss_mae=0
# all_predictions = []
# all_targets = []
# with torch.no_grad():
#     for data, target in test_loader:
#         data, target = data.to(device), target.to(device)
        
#         target = target.squeeze(-1)  # 调整目标尺寸
#         output = model(data)
#         test_loss += criterion(output, target).item()
#         test_loss_mae+=criterion_mae(output, target).item()
#         all_predictions.append(output.cpu().numpy())
#         all_targets.append(target.cpu().numpy())

# test_loss /= len(test_loader)
# test_loss_mae/=len(test_loader)
# print(f'Test Loss: {test_loss}')
# print(f'Test Loss mae: {test_loss_mae}')

### 从磁盘上load

In [106]:
model=TransformerEncoderModel().to(device)
model.load_state_dict(torch.load('../model/T2/TransformerEncoder.pth'))

<All keys matched successfully>

In [108]:
criterion=nn.L1Loss()
# 在测试集上验证模型并输出预测标签
model.eval()
test_loss = 0
all_predictions = []
all_targets = []
with torch.no_grad():
    for data, target in test_loader:
        # data:[16, 8, 9] target:[16, 4, 1],output:[16,1]
        for i in range(output_nums):
            data, target = data.to(device), target.to(device)
            target = target.squeeze(-1)
            output = model(data)
            # 将output变为[16,1,1]的形式
            output=output.unsqueeze(1)
            # output第一维加上其他特征，其他特征分别是下一个data的第2维到第9维
            




            # 将data的第1个维度集体向左平移，再加上output，例如[1,2,3,4,5,6,7,8]->[2,3,4,5,6,7,8,prediction]
            data=torch.cat((data[:,1:,:],output),1)
            print(data)
            print(output)
            test_loss += criterion(output, target).item()
            all_predictions.append(output.cpu().numpy())
            all_targets.append(target.cpu().numpy())
            break


        break


test_loss /= len(test_loader)
print(f'Test Loss: {test_loss}')



RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 9 but got size 1 for tensor number 1 in the list.

### 可视化分别看1步-4步